In [17]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from functools import partial
import sys
sys.path.append('/home/tomek/ib_tools')
from indicators import get_ATR, get_signals
%matplotlib inline

In [18]:
df = pd.read_csv('/home/tomek/ib_tools/blotter/backtest_2019-11-07_17-05.csv')
df.time = df.time.astype('datetime64')
df.set_index('time', inplace=True)
df

,sys_time,contract,action,amount,price,exec_ids,order_id,reason,com_exec_id,commission,currency,realizedPNL,com_sys_time
time,,,,,,,,,,,,,
2018-02-09 00:50:00,2019-11-07 17:08:06.388268,NQZ9,SELL,1,6304.175,[1],1,entry,1,1.3,USD,0.0,2019-11-07 17:08:06.388621
2018-02-09 00:58:00,2019-11-07 17:08:06.420664,NKDZ9,SELL,1,21102.500,[2],3,entry,2,1.3,USD,0.0,2019-11-07 17:08:06.420898
2018-02-09 01:12:00,2019-11-07 17:08:06.511802,GCZ9,BUY,1,1323.830,[3],5,entry,3,1.3,USD,0.0,2019-11-07 17:08:06.512038
2018-02-09 01:15:00,2019-11-07 17:08:06.513828,NQZ9,BUY,1,6346.150,[4],2,stop-out,4,1.3,USD,-839.5,2019-11-07 17:08:06.513884
2018-02-09 01:16:00,2019-11-07 17:08:06.514725,NKDZ9,BUY,1,21230.000,[5],4,stop-out,5,1.3,USD,-637.5,2019-11-07 17:08:06.514772
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-01 13:37:00,2019-11-07 20:43:27.790208,CLZ9,BUY,1,54.842,[7929],9082,entry,7929,1.3,USD,0.0,2019-11-07 20:43:27.791955
2019-11-01 13:39:00,2019-11-07 20:43:28.013105,ESZ9,BUY,1,3048.725,[7930],9084,entry,7930,1.3,USD,0.0,2019-11-07 20:43:28.014858
2019-11-01 14:36:00,2019-11-07 20:43:31.996491,YM DEC 19,BUY,1,27132.100,[7931],9086,entry,7931,1.3,USD,0.0,2019-11-07 20:43:31.998295


In [19]:
df[df.reason == 'entry'].price.count()

3962

In [20]:
df[df.reason != 'entry'].price.count()

3971

In [21]:
df.price.count()

7933

In [22]:
df[df.realizedPNL > 4000]

,sys_time,contract,action,amount,price,exec_ids,order_id,reason,com_exec_id,commission,currency,realizedPNL,com_sys_time
time,,,,,,,,,,,,,
2018-03-02 18:45:00,2019-11-07 17:10:24.350339,YM DEC 19,BUY,1,24473.000000,[319],364,close,319,1.3,USD,4156.50,2019-11-07 17:10:24.350587
2018-03-05 01:23:00,2019-11-07 17:10:28.206211,NKDZ9,BUY,1,21027.995215,[326],372,close,326,1.3,USD,4433.57,2019-11-07 17:10:28.206454
2018-06-07 16:40:00,2019-11-07 17:25:15.172695,NQZ9,SELL,1,7176.699330,[1540],1772,close,1540,1.3,USD,4014.49,2019-11-07 17:25:15.173343
2018-06-29 04:15:00,2019-11-07 17:29:33.119293,CLZ9,SELL,1,73.122000,[1767],2029,close,1767,1.3,USD,4708.00,2019-11-07 17:29:33.120062
2018-10-03 06:14:00,2019-11-07 17:50:05.942346,NKDZ9,SELL,1,24123.500000,[2785],3177,close,2785,1.3,USD,7110.00,2019-11-07 17:50:05.943254
2018-10-11 15:33:00,2019-11-07 17:52:55.434288,NQZ9,BUY,1,7072.075000,[2905],3320,close,2905,1.3,USD,6349.00,2019-11-07 17:52:55.435288
2018-10-11 16:00:00,2019-11-07 17:52:57.496716,ESZ9,BUY,1,2795.250000,[2907],3322,close,2907,1.3,USD,4356.25,2019-11-07 17:52:57.497827
2018-11-14 10:29:00,2019-11-07 18:04:03.444395,CLZ9,BUY,1,56.463949,[3461],3955,close,3461,1.3,USD,4062.18,2019-11-07 18:04:03.445534
2018-11-20 16:57:00,2019-11-07 18:06:13.941517,NQZ9,BUY,1,6606.150000,[3551],4053,close,3551,1.3,USD,4284.50,2019-11-07 18:06:13.942756


In [23]:
df.tail(50)

,sys_time,contract,action,amount,price,exec_ids,order_id,reason,com_exec_id,commission,currency,realizedPNL,com_sys_time
time,,,,,,,,,,,,,
2019-10-30 15:06:00,2019-11-07 20:41:44.166899,ESZ9,SELL,1,3030.050000,[7884],9031,close,7884,1.3,USD,608.75,2019-11-07 20:41:44.169465
2019-10-30 15:33:00,2019-11-07 20:41:46.085738,CLZ9,SELL,1,55.014000,[7885],9032,entry,7885,1.3,USD,0.00,2019-11-07 20:41:46.087522
2019-10-30 16:33:00,2019-11-07 20:41:49.580667,YM DEC 19,SELL,1,26991.900000,[7886],9034,close,7886,1.3,USD,545.50,2019-11-07 20:41:49.583053
2019-10-30 16:57:00,2019-11-07 20:41:50.759428,ZB DEC 19,BUY,1,159.531250,[7887],9035,entry,7887,1.3,USD,0.00,2019-11-07 20:41:50.761263
2019-10-30 19:00:00,2019-11-07 20:41:55.855984,ZB DEC 19,SELL,1,159.190625,[7888],9036,stop-out,7888,1.3,USD,-340.62,2019-11-07 20:41:55.856073
2019-10-30 19:02:00,2019-11-07 20:41:56.015920,GCZ9,SELL,1,1494.580000,[7889],9037,close,7889,1.3,USD,1.00,2019-11-07 20:41:56.018295
2019-10-30 19:17:00,2019-11-07 20:41:57.349937,NQZ9,BUY,1,8069.025000,[7890],9038,entry,7890,1.3,USD,0.00,2019-11-07 20:41:57.351742
2019-10-30 19:31:00,2019-11-07 20:41:58.409876,NQZ9,SELL,1,8051.050000,[7891],9039,stop-out,7891,1.3,USD,-359.50,2019-11-07 20:41:58.409960
2019-10-30 19:32:00,2019-11-07 20:41:58.550527,GCZ9,SELL,1,1489.200000,[7892],9040,entry,7892,1.3,USD,0.00,2019-11-07 20:41:58.552311


In [24]:
df.loc['20190829':].head(50)

,sys_time,contract,action,amount,price,exec_ids,order_id,reason,com_exec_id,commission,currency,realizedPNL,com_sys_time
time,,,,,,,,,,,,,
2019-08-29 00:01:00,2019-11-07 20:05:16.341629,ZB DEC 19,SELL,1,165.562195,[7052],8083,close,7052,1.3,USD,450.91,2019-11-07 20:05:16.343800
2019-08-29 03:10:00,2019-11-07 20:05:20.101978,NQZ9,SELL,1,7568.950000,[7053],8082,stop-out,7053,1.3,USD,-549.50,2019-11-07 20:05:20.102062
2019-08-29 07:53:00,2019-11-07 20:05:25.975437,GCZ9,BUY,1,1554.460000,[7054],8084,entry,7054,1.3,USD,0.00,2019-11-07 20:05:25.977008
2019-08-29 09:13:00,2019-11-07 20:05:28.032320,NKDZ9,BUY,1,20551.500000,[7055],8064,stop-out,7055,1.3,USD,-685.00,2019-11-07 20:05:28.032406
2019-08-29 09:14:00,2019-11-07 20:05:28.160703,NQZ9,BUY,1,7655.175000,[7056],8086,entry,7056,1.3,USD,0.00,2019-11-07 20:05:28.162403
2019-08-29 09:17:00,2019-11-07 20:05:28.482678,GEZ9,SELL,1,98.157500,[7057],8088,entry,7057,1.3,USD,0.00,2019-11-07 20:05:28.484266
2019-08-29 09:25:00,2019-11-07 20:05:28.846787,GEZ9,BUY,1,98.165000,[7058],8089,stop-out,7058,1.3,USD,-18.75,2019-11-07 20:05:28.846856
2019-08-29 09:51:00,2019-11-07 20:05:29.617508,NKDZ9,BUY,1,20580.000000,[7059],8090,entry,7059,1.3,USD,0.00,2019-11-07 20:05:29.619105
2019-08-29 09:56:00,2019-11-07 20:05:29.699804,GCZ9,SELL,1,1551.180000,[7060],8085,stop-out,7060,1.3,USD,-328.00,2019-11-07 20:05:29.699875


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7933 entries, 2018-02-09 00:50:00 to 2019-11-01 15:05:00
Data columns (total 13 columns):
sys_time        7933 non-null object
contract        7933 non-null object
action          7933 non-null object
amount          7933 non-null int64
price           7933 non-null float64
exec_ids        7933 non-null object
order_id        7933 non-null int64
reason          7933 non-null object
com_exec_id     7933 non-null int64
commission      7933 non-null float64
currency        7933 non-null object
realizedPNL     7933 non-null float64
com_sys_time    7933 non-null object
dtypes: float64(3), int64(3), object(7)
memory usage: 1.2+ MB


In [26]:
df.realizedPNL.sum()

76804.23000000001

In [27]:
# percentage of winning transactions
round(df[df.realizedPNL>0].realizedPNL.count() / df[df.realizedPNL != 0].realizedPNL.count(), 2)

0.23

In [28]:
# avg win, avg lose, ratio
avg_win = round(df[df.realizedPNL > 0].realizedPNL.mean())
avg_lose = round(df[df.realizedPNL < 0].realizedPNL.mean())
ratio = round(abs(avg_win / avg_lose), 2)
avg_win, avg_lose, ratio

(1004.0, -273.0, 3.68)

In [29]:
df.realizedPNL.min(), df.realizedPNL.max(), round(abs(df.realizedPNL.max()/df.realizedPNL.min()))

(-1696.0, 7842.5, 5.0)

In [30]:
df.sort_values('realizedPNL').iloc[1]

sys_time        2019-11-07 19:32:22.381043
contract                              CLZ9
action                                 BUY
amount                                   1
price                               59.716
exec_ids                            [6190]
order_id                              7080
reason                            stop-out
com_exec_id                           6190
commission                             1.3
currency                               USD
realizedPNL                          -1287
com_sys_time    2019-11-07 19:32:22.381121
Name: 2019-07-01 00:00:00, dtype: object

In [31]:
df.loc['20181011 20:00:00':].head(50)

,sys_time,contract,action,amount,price,exec_ids,order_id,reason,com_exec_id,commission,currency,realizedPNL,com_sys_time
time,,,,,,,,,,,,,
2018-10-11 20:28:00,2019-11-07 17:53:11.141732,ZF DEC 19,BUY,1,112.246875,[2916],3331,entry,2916,1.3,USD,0.00,2019-11-07 17:53:11.142679
2018-10-11 20:38:00,2019-11-07 17:53:11.587253,NQZ9,SELL,1,6995.750000,[2917],3333,entry,2917,1.3,USD,0.00,2019-11-07 17:53:11.588331
2018-10-11 20:39:00,2019-11-07 17:53:11.786977,ESZ9,SELL,1,2742.200000,[2918],3335,entry,2918,1.3,USD,0.00,2019-11-07 17:53:11.787793
2018-10-11 20:39:00,2019-11-07 17:53:11.787829,YM DEC 19,SELL,1,25178.300000,[2919],3336,entry,2919,1.3,USD,0.00,2019-11-07 17:53:11.788493
2018-10-11 20:39:00,2019-11-07 17:53:11.788525,NKDZ9,SELL,1,22473.500000,[2920],3337,entry,2920,1.3,USD,0.00,2019-11-07 17:53:11.789068
2018-10-11 21:16:00,2019-11-07 17:53:14.324492,NQZ9,BUY,1,7035.375000,[2921],3334,stop-out,2921,1.3,USD,-792.50,2019-11-07 17:53:14.324548
2018-10-11 21:17:00,2019-11-07 17:53:14.394496,ESZ9,BUY,1,2757.700000,[2922],3338,stop-out,2922,1.3,USD,-775.00,2019-11-07 17:53:14.394558
2018-10-11 21:17:00,2019-11-07 17:53:14.394654,YM DEC 19,BUY,1,25302.700000,[2923],3339,stop-out,2923,1.3,USD,-622.00,2019-11-07 17:53:14.394694
2018-10-11 21:22:00,2019-11-07 17:53:14.788924,NKDZ9,BUY,1,22620.500000,[2924],3340,stop-out,2924,1.3,USD,-735.00,2019-11-07 17:53:14.789019


In [32]:
df.realizedPNL.resample('M').sum()

time
2018-02-28   -13689.47
2018-03-31    20740.87
2018-04-30    -7580.60
2018-05-31    -6224.06
2018-06-30     8777.03
2018-07-31    -7930.60
2018-08-31     6571.12
2018-09-30    -7138.61
2018-10-31    -1270.28
2018-11-30     4058.51
2018-12-31    50264.30
2019-01-31    -3130.07
2019-02-28     6120.51
2019-03-31     2355.74
2019-04-30    11219.13
2019-05-31   -18710.75
2019-06-30    23151.22
2019-07-31    -8151.26
2019-08-31    14284.65
2019-09-30    -1376.42
2019-10-31     2034.83
2019-11-30     2428.44
Freq: M, Name: realizedPNL, dtype: float64